In [1]:
import sys
sys.path.append("/home/rishabhtiwari")
import torch
from QuantSpec_magidec.Engine.backend_quantspec import LMBackend
from pathlib import Path

checkpoint_path = Path("/home/rishabhtiwari/cache/checkpoints/meta-llama/Llama-2-7b-hf/model.pth")
DTYPE = torch.bfloat16
DEVICE = 'cuda:2'
target_dec_list = [1]
draft_dec_list = [1]
engine = LMBackend(dtype=DTYPE, device=DEVICE, dec_list=target_dec_list, draft_dec_list=draft_dec_list)
engine.load_model(checkpoint_path, use_tp=False, rank_group = [0], group=None, quantize=True, marlin_checkpoint="/home/rishabhtiwari/marlin2/marlin/gptq/checkpoint.pt.marlin.g128")


/home/rishabhtiwari/QuantSpec_magidec/Engine/utils.py:79: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("mylib::flash_verification")
/home/rishabhtiwari/QuantSpec_magidec/Engine/utils.py:171: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("mylib::flash_decoding")
/home/rishabhtiwari/QuantSpec_magidec/Engine/utils.py:208: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("mylib::custom_func")
/home/rishabhtiwari/QuantSpec_magidec/Engine/utils.py:223: FutureWarning: `t

In [2]:
print(engine.model.layers[0].feed_forward.w1_quantized.s.shape)
print(engine.model.layers[0].feed_forward.w2_quantized.s.shape)
print(engine.model.layers[0].feed_forward.w3_quantized.s.shape)


torch.Size([32, 11008])
torch.Size([86, 4096])
torch.Size([32, 11008])


In [2]:
# import marlin
# l = marlin.Layer(4096, 14336, 128)
# # l.pack(torch.randn(1, 4096, 14336), torch.randn(1, 14336, 4096))

In [3]:
engine.model.eval()

Transformer(
  (tok_embeddings): Embedding(32000, 4096)
  (layers): ModuleList(
    (0-31): 32 x TransformerBlock(
      (attention): Attention(
        (wqkv): Linear(in_features=4096, out_features=12288, bias=False)
        (wo): Linear(in_features=4096, out_features=4096, bias=False)
      )
      (feed_forward): FeedForward(
        (w1): Linear(in_features=4096, out_features=11008, bias=False)
        (w3): Linear(in_features=4096, out_features=11008, bias=False)
        (w2): Linear(in_features=11008, out_features=4096, bias=False)
        (w1_quantized): Layer()
        (w3_quantized): Layer()
        (w2_quantized): Layer()
      )
      (ffn_norm): RMSNorm()
      (attention_norm): RMSNorm()
    )
  )
  (norm): RMSNorm()
  (output): Linear(in_features=4096, out_features=32000, bias=False)
)

In [6]:
random_input = torch.randn(1, 1, 11008).to(torch.bfloat16).to('cuda:2')
engine.model.layers[0].feed_forward.w2_quantized(random_input).shape

torch.Size([1, 1, 4096])

In [8]:
engine.model.layers[0].feed_forward.w3_quantized.workspace.shape

torch.Size([1376])

In [4]:
14336 // 128 * 16

1792

In [2]:
import marlin
marlin.Linear()

AttributeError: module 'marlin' has no attribute 'Linear'

In [2]:
checkpoint_path.parent.name

'Meta-Llama-3.1-8B'

In [5]:
engine.model.layers[0].feed_forward.w1_quantized

Layer()

In [2]:
engine.model.to('cuda:0')


Transformer(
  (tok_embeddings): Embedding(128256, 4096)
  (layers): ModuleList(
    (0-31): 32 x TransformerBlock(
      (attention): Attention(
        (wqkv): Linear(in_features=4096, out_features=6144, bias=False)
        (wo): Linear(in_features=4096, out_features=4096, bias=False)
      )
      (feed_forward): FeedForward(
        (w1): Linear(in_features=4096, out_features=14336, bias=False)
        (w3): Linear(in_features=4096, out_features=14336, bias=False)
        (w2): Linear(in_features=14336, out_features=4096, bias=False)
        (w1_quantized): Layer()
        (w3_quantized): Layer()
        (w2_quantized): Layer()
      )
      (ffn_norm): RMSNorm()
      (attention_norm): RMSNorm()
    )
  )
  (norm): RMSNorm()
  (output): Linear(in_features=4096, out_features=128256, bias=False)
)

In [27]:
import torch.nn as nn
import marlin
def replace_linear(module, name_filter=lambda n: True, groupsize=-1, name=''):
    """Recursively replace all `torch.nn.Linear` layers by empty Marlin layers.
    @module: top-level module in which to perform the replacement 
    @name_filter: lambda indicating if a layer should be replaced
    @groupsize: marlin groupsize
    @name: root-level name
    """
    if isinstance(module, marlin.Layer):
        return
    for attr in dir(module):
        tmp = getattr(module, attr)
        name1 = name + '.' + attr if name != '' else attr
        if isinstance(tmp, nn.Linear) and name_filter(name1):
            setattr(
                module, attr, marlin.Layer(tmp.in_features, tmp.out_features, groupsize=groupsize)
            )
    for name1, child in module.named_children():
        replace_linear(child, name_filter, groupsize=groupsize, name=name + '.' + name1 if name != '' else name1)

    

In [28]:
replace_linear(engine.model, name_filter=lambda n: 'feed_forward' in n)

FeedForward(
  (w1): Layer()
  (w3): Layer()
  (w2): Layer()
)

In [30]:
marlin_dict = torch.load("/home/rishabhtiwari/cache/llama3_8b_checkpoint.pt.marlin.g128")

/tmp/ipykernel_275405/878203640.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  marlin_dict = torch.load("/home/rishabhtiwari/cache/llama3_8b_checkpoint.pt.marlin.g128")

In [31]:
marlin_dict.keys()

odict_keys(['model.embed_tokens.weight', 'model.layers.0.self_attn.q_proj.B', 'model.layers.0.self_attn.q_proj.s', 'model.layers.0.self_attn.k_proj.B', 'model.layers.0.self_attn.k_proj.s', 'model.layers.0.self_attn.v_proj.B', 'model.layers.0.self_attn.v_proj.s', 'model.layers.0.self_attn.o_proj.B', 'model.layers.0.self_attn.o_proj.s', 'model.layers.0.mlp.gate_proj.B', 'model.layers.0.mlp.gate_proj.s', 'model.layers.0.mlp.up_proj.B', 'model.layers.0.mlp.up_proj.s', 'model.layers.0.mlp.down_proj.B', 'model.layers.0.mlp.down_proj.s', 'model.layers.0.input_layernorm.weight', 'model.layers.0.post_attention_layernorm.weight', 'model.layers.1.self_attn.q_proj.B', 'model.layers.1.self_attn.q_proj.s', 'model.layers.1.self_attn.k_proj.B', 'model.layers.1.self_attn.k_proj.s', 'model.layers.1.self_attn.v_proj.B', 'model.layers.1.self_attn.v_proj.s', 'model.layers.1.self_attn.o_proj.B', 'model.layers.1.self_attn.o_proj.s', 'model.layers.1.mlp.gate_proj.B', 'model.layers.1.mlp.gate_proj.s', 'model.l

In [37]:
engine.model.layers[0].feed_forward.w1.B

tensor([[ -427395805, -2041014920, -1522833463,  ...,  1969854105,
          -943142709,  1234729076],
        [   80444266,  1954973867,  -160643367,  ...,  1500162713,
           882283418,  1731696187],
        [ 2110164110,  1718056282, -1967593290,  ..., -1882092933,
         -1252493735,  1498966756],
        ...,
        [-1737259831,  2044893053,  -544820389,  ..., -1469539465,
          -915895163,  -377911905],
        [ 2054572453, -1687630509, -1470728552,  ...,   691583303,
          1773821270,  1891409297],
        [-1409517093, -1977844776, -1432516168,  ..., -1738298749,
         -2037937471,  2016004218]], dtype=torch.int32)

In [36]:
# Map marlin dict keys to engine model keys by replacing feed_forward with mlp
for key in marlin_dict.keys():
    if 'mlp' in key:
        # Get the corresponding engine model key by replacing mlp with feed_forward
        engine_key = key.replace('mlp', 'feed_forward')
        
        # Map the gate_proj, up_proj, down_proj to w1, w2, w3 respectively
        if 'gate_proj' in key:
            engine_key = engine_key.replace('gate_proj', 'w1_quantized')
        elif 'up_proj' in key:
            engine_key = engine_key.replace('up_proj', 'w2_quantized') 
        elif 'down_proj' in key:
            engine_key = engine_key.replace('down_proj', 'w3_quantized')
            
        # Get the layer components from the key
        parts = engine_key.split('.')
        layer_idx = int(parts[2])
        
        # Get the corresponding module
        layer = engine.model.layers[layer_idx].feed_forward
        module = getattr(layer, parts[-2])
        
        # Load the weights
        param_name = parts[-1] # B or s
        setattr(module, param_name, marlin_dict[key])


In [38]:
parts[-2]

'w3'

In [16]:
ls /home/xihc/marlin/gptq/llama3_8b_checkpoint.pt.marlin.g128

__pycache__/                          llama2_70b_checkpoint.pt.marlin.g128
datautils.py                          llama2_7b_checkpoint.pt.marlin.g128
eval.py                               llama3_70b_checkpoint.pt.marlin.g128
gptq.py                               llama3_8b_checkpoint.pt.marlin.g128
llama2.py                             quant.py
llama2_13b_checkpoint.pt.marlin.g128  test_llama_load.py


In [25]:
def print_children(module, name=''):
    for name1, child in module.named_children():
        print(name + '.' + name1 if name != '' else name1)
        print_children(child, name + '.' + name1 if name != '' else name1)
print_children(engine.model)


tok_embeddings
layers
layers.0
layers.0.attention
layers.0.attention.wqkv
layers.0.attention.wo
layers.0.feed_forward
layers.0.feed_forward.w1
layers.0.feed_forward.w3
layers.0.feed_forward.w2
layers.0.ffn_norm
layers.0.attention_norm
layers.1
layers.1.attention
layers.1.attention.wqkv
layers.1.attention.wo
layers.1.feed_forward
layers.1.feed_forward.w1
layers.1.feed_forward.w3
layers.1.feed_forward.w2
layers.1.ffn_norm
layers.1.attention_norm
layers.2
layers.2.attention
layers.2.attention.wqkv
layers.2.attention.wo
layers.2.feed_forward
layers.2.feed_forward.w1
layers.2.feed_forward.w3
layers.2.feed_forward.w2
layers.2.ffn_norm
layers.2.attention_norm
layers.3
layers.3.attention
layers.3.attention.wqkv
layers.3.attention.wo
layers.3.feed_forward
layers.3.feed_forward.w1
layers.3.feed_forward.w3
layers.3.feed_forward.w2
layers.3.ffn_norm
layers.3.attention_norm
layers.4
layers.4.attention
layers.4.attention.wqkv
layers.4.attention.wo
layers.4.feed_forward
layers.4.feed_forward.w1
layer

In [7]:
checkpoint_path.parent

PosixPath('/home/rishabhtiwari/cache/checkpoints/meta-llama/Meta-Llama-3.1-8B')

In [1]:
import torch
query_sequence_length = 1
verify_len = 1
max_residual_len = 10
residual_len = 5

attn_mask = torch.arange(max_residual_len).unsqueeze(0).repeat(query_sequence_length, 1)
attn_mask -= torch.arange(query_sequence_length).unsqueeze(1).repeat(1, max_residual_len)
attn_mask = (attn_mask <= residual_len-query_sequence_length)*1.
attn_mask

tensor([[1., 1., 1., 1., 1., 0., 0., 0., 0., 0.]])

In [4]:
attn_mask2 = torch.tril(torch.ones((verify_len, max_residual_len)), diagonal=residual_len-verify_len)
attn_mask2

tensor([[1., 1., 1., 1., 1., 0., 0., 0., 0., 0.]])

In [5]:
attn_mask2 == attn_mask

tensor([[True, True, True, True, True, True, True, True, True, True]])

In [1]:
import torch

def draft_update(k_cache, v_cache, k, v, cache_seqlens, qcache_seqlens):
    # Create indices for scatter operation
    bsz, num_heads, update_len, head_dim = k.shape
    seq_len = k_cache.shape[2]
    
    # Create index tensor for scatter
    indices = torch.arange(update_len, device=k.device)
    indices = indices.view(1, 1, -1, 1)  # [1, 1, update_len, 1]
    indices = indices + qcache_seqlens.view(-1, 1, 1, 1)  # [bsz, 1, update_len, 1]
    indices = indices.expand(bsz, num_heads, update_len, head_dim)
    print(indices)
    
    # Perform scatter operations
    k_cache.scatter_(dim=2, index=indices, src=k)
    v_cache.scatter_(dim=2, index=indices, src=v)
    
    return k_cache, v_cache
    
    

In [ ]:

def draft_update(k_cache, v_cache, k, v, cache_seqlens, qcache_seqlens):
    bsz = k_cache.shape[0]
    assert bsz == 1, "Batch size > 1 not supported yet"
    
    # Calculate start index for the current batch
    start_idx = qcache_seqlens[0]  # Using only first batch element since bsz=1
    seq_len = k_cache.shape[2]
    
    # Create a mask for valid positions
    # This avoids data-dependent indexing by operating on the full tensor
    position_indices = torch.arange(self.k_cache.shape[2], device=k_cache.device)
    valid_positions = (position_indices >= start_idx) & (position_indices < start_idx + seq_len)
    
    # Expand mask to match cache dimensions
    mask = valid_positions.view(1, 1, -1, 1).expand_as(self.k_cache[:, :, :, :])
    
    # Create expanded k_cache and v_cache
    expanded_k = k_cache.unsqueeze(2).expand(-1, -1, self.k_cache.shape[2], -1)
    expanded_v = v_cache.unsqueeze(2).expand(-1, -1, self.v_cache.shape[2], -1)
    
    # Use masked operations instead of direct indexing
    self.k_cache = torch.where(mask, expanded_k, self.k_cache)
    self.v_cache = torch.where(mask, expanded_v, self.v_cache)

In [2]:
def test_draft_update():
    # Create test inputs
    batch_size = 2
    num_heads = 4
    seq_len = 8
    head_dim = 16
    
    # Create sample caches
    k_cache = torch.zeros(batch_size, num_heads, seq_len, head_dim)
    v_cache = torch.zeros(batch_size, num_heads, seq_len, head_dim)
    k = torch.randn(batch_size, num_heads, 1, head_dim)
    v = torch.randn(batch_size, num_heads, 1, head_dim)
    
    # Create sample sequence lengths
    cache_seqlens = torch.tensor([6, 4])  # Different lengths for each batch
    qcache_seqlens = torch.tensor([2, 1])  # Different starting points
    
    # Keep copies of original caches
    k_cache_orig = k_cache.clone()
    v_cache_orig = v_cache.clone()
    
    # Run the function
    k_out, v_out = draft_update(k_cache, v_cache,k,v, cache_seqlens, qcache_seqlens)
    
    # Verify shapes haven't changed
    assert k_out.shape == k_cache_orig.shape
    assert v_out.shape == v_cache_orig.shape
    
    # Verify the updates happened at correct positions
    for b in range(batch_size):
        start_idx = qcache_seqlens[b]
        end_idx = qcache_seqlens[b] + k.shape[2]
        
        # Check that values were copied correctly
        assert torch.allclose(k_out[b, :, start_idx:end_idx], k[b])
        assert torch.allclose(v_out[b, :, start_idx:end_idx], v[b])
        
        # Check that values outside update range remain unchanged
        if start_idx > 0:
            assert torch.allclose(k_out[b, :, :start_idx], k_cache_orig[b, :, :start_idx])
            assert torch.allclose(v_out[b, :, :start_idx], v_cache_orig[b, :, :start_idx])
        if end_idx < seq_len:
            assert torch.allclose(k_out[b, :, end_idx:], k_cache_orig[b, :, end_idx:])
            assert torch.allclose(v_out[b, :, end_idx:], v_cache_orig[b, :, end_idx:])
    
    print("All tests passed!")

# Run the test
test_draft_update()


tensor([[[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]],

         [[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]],

         [[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]],

         [[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]]],


        [[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

         [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

         [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

         [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]]])
All tests passed!


In [39]:
bsz = 2
seq_len = 5

mask = torch.arange(seq_len).unsqueeze(0) >= index_list.unsqueeze(1)
mask &= torch.arange(seq_len).unsqueeze(0) < (index_list + a.shape[1]).unsqueeze(1)
mask = torch.tensor(mask)
mask

/tmp/ipykernel_3847217/3799017049.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask = torch.tensor(mask)


tensor([[False,  True,  True, False, False],
        [False, False,  True,  True, False]])

tensor([[ 1.5959],
        [ 0.0472],
        [-0.8919],
        [-0.2357]])

In [40]:
A[mask] = a.view(-1,a.shape[-1])

In [41]:
A

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000],
         [ 0.2664, -1.1244,  0.1311,  1.5670, -0.5227, -0.0628, -0.3672,
           0.7776, -0.6443,  0.5735],
         [ 1.2952, -1.7487,  0.5188, -0.9836,  0.3290,  0.1861,  1.1023,
          -1.0282,  0.1419,  0.3721],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000],
         [ 2.0867, -1.0878, -1.5198, -0.6441,  1.2848,  0.5716, -0.1964,
          -1.4325,  0.4747,  0.7528],
         [ 0.6980, -0.2796,  0.1953,  0.0589,  1.1897,  0.2222,  0.0208,
          -1.0235,  0.4140, -1.9478],

In [43]:
a.shape

torch.Size([2, 2, 10])

In [ ]:
A

In [45]:
A = torch.zeros(2, 5, 10)
# a = torch.randn(2, 2, 10)

index_list = torch.tensor([1, 2]) # 2, 2, 5
A[0, 1:1+a.shape[1], :] = a[0] 
A[1, 2:2+a.shape[1], :] = a[1]
A

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000],
         [ 0.2664, -1.1244,  0.1311,  1.5670, -0.5227, -0.0628, -0.3672,
           0.7776, -0.6443,  0.5735],
         [ 1.2952, -1.7487,  0.5188, -0.9836,  0.3290,  0.1861,  1.1023,
          -1.0282,  0.1419,  0.3721],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000],
         [ 2.0867, -1.0878, -1.5198, -0.6441,  1.2848,  0.5716, -0.1964,
          -1.4325,  0.4747,  0.7528],
         [ 0.6980, -0.2796,  0.1953,  0.0589,  1.1897,  0.2222,  0.0208,
          -1.0235,  0.4140, -1.9478],

In [16]:
index=torch.tensor([1, 2]).view(1, 2, 1).repeat(2,1,5)
index.shape

torch.Size([2, 2, 5])

In [19]:
A.scatter_(dim=1, index=index, src=a)
print(A)

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [-0.2241, -1.1241, -1.8280, -0.4621,  0.3573],
         [-0.3164,  0.9304,  0.2137, -0.8889, -1.4511],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [-0.4408,  0.6711,  3.1076, -0.6692,  0.4591],
         [-0.6139,  0.2703,  0.3493, -2.3580, -0.8639],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000, 

In [20]:
A[0]

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.2241, -1.1241, -1.8280, -0.4621,  0.3573],
        [-0.3164,  0.9304,  0.2137, -0.8889, -1.4511],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])

In [55]:
x = torch.zeros(5, 3)
t = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=torch.float)
index = torch.tensor([0, 4, 2])
# x.index_copy_(0, index, t)
# tensor([[ 1.,  2.,  3.],
#         [ 0.,  0.,  0.],
#         [ 7.,  8.,  9.],
#         [ 0.,  0.,  0.],
#         [ 4.,  5.,  6.]])


In [57]:
t, t.shape

(tensor([[1., 2., 3.],
         [4., 5., 6.],
         [7., 8., 9.]]),
 torch.Size([3, 3]))

In [58]:
x, x.shape

(tensor([[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]),
 torch.Size([5, 3]))